# Nonce reuse attack

Given two signatures $(r, s)$ and $(r, s')$ on two different messages $m$ and $m'$ that share the same nonce $k$ (hence also $r$) we can easily compute the private key.

### Signature 1

\begin{align}
r &\equiv ([k]G)_x\\
s &\equiv k^{-1} (H(m) + r d)
\end{align}


### Signature 2

\begin{align}
r &\equiv ([k]G)_x\\
s' &\equiv k^{-1} (H(m') + r d)
\end{align}


### Key recovery

\begin{align}
s &\equiv k^{-1} (H(m) + r d)\\
s' &\equiv k^{-1} (H(m') + r d)
\end{align}


Now subtract the equations from each other.

$$ s - s' \equiv k^{-1} (H(m) - H(m')) $$


Then compute $k$.

$$ k \equiv \frac{H(m) - H(m')}{s - s'} $$

Pick an equation and recover the private key $d$.

$$ d \equiv \frac{s k - H(m)}{r} $$

In [ ]:
from hashlib import sha1
from pyecsca.ec.params import get_params
from pyecsca.ec.mod import Mod

from client import DeviceTarget

In [ ]:
curve = get_params("nist", "P-192", "projective")

In [ ]:
target = DeviceTarget()

target.timeout = 2000
base_freq = 15000000
target.scope.io.clkout = base_freq
target.scope.adc.clk_freq = base_freq
target.scope.adc.samples = 50000

In [ ]:
target.flash("../micro-ecc-CWNANO.hex")

In [ ]:
target.connect()
target.init_prng(bytes.fromhex("cafebabe"))
target.generate_keypair()
pubkey = target.export()

msg1 = b"This is some text"
hash1 = sha1(msg1).digest()
target.init_prng(bytes.fromhex("deadbeef"))
signature1 = target.sign(hash1)

msg2 = b"This is another text"
hash2 = sha1(msg2).digest()
target.init_prng(bytes.fromhex("deadbeef"))
signature2 = target.sign(hash2)

target.halt()
target.disconnect()

In [ ]:
n = curve.order
h1 = Mod(int.from_bytes(hash1, byteorder="big"), n)
h2 = Mod(int.from_bytes(hash2, byteorder="big"), n)
r1 = Mod(signature1[0], n)
s1 = Mod(signature1[1], n)
s2 = Mod(signature2[1], n)

k = (h1 - h2) / (s1 - s2)

privkey = (s1 * k - h1) / r1

In [ ]:
computed_pubkey = curve.curve.affine_multiply(curve.generator.to_affine(), int(privkey))

In [ ]:
print(computed_pubkey)
print(pubkey)